-------

## 12-16

### 10-12

1. 面试
2. 会议
3. 一键配置: 自研开发, 耗时好力, 扩展功能灵活
4. 一键配置: OpenStack是一个开源的云计算管理平台, 标准统一, 大规模
5. 一键配置: 滴滴云快照, 快捷省事, 会产生一点费用 (推荐使用)
6. 远程监控: plesk专业主机管理软件, 面向Web服务, 不免费
7. 远程监控: StatsD, serverstatus 都是notejs方面的
8. 远程监控: Nagios 监控系统, 商业软件, 免费监控7个终端
9. 远程监控: Shenkin 重新实现Nagios免费, 相对复杂, 插件较多, 功能繁琐

验证两台主机信任是否已成功建立

Nagios的主要功能特点：
- 监视网络服务 (SMTP, POP3, HTTP, NNTP, PING等)
- 监视主机资源 (进程, 磁盘等)
- 简单的插件设计可以轻松扩展Nagios的监视功能
- 服务等监视的并发处理
- 错误通知功能 (通过email, pager, 或其他用户自定义方法)
- 可指定自定义的事件处理控制器
- 可选的基于浏览器的WEB界面以方便系统管理人员查看网络状态，各种系统问题，以及日志等等
- 可以通过手机查看系统监控信息

### 10-13

1. 一键配置滴滴云快照方式在P4机器验证可行
2. consul 官方文档及软件功能分析
  - 服务发现
  - 健康检测
  - 键值存储
  - 安全服务通信
  - 多数据中心
  
  (文档中没有发现有关控制的功能, 欠缺灵活性, 扩展不便, 所以及时上报异常状态和控制服务的功能需要单独设计) 
    
3. 登陆认证
- 数据传输安全(TLS)
- 还需考虑整机下线

(远程监控方案: 采用consul的检测 + 实现上报/控制)

https://www.consul.io/docs/intro

Consul使用gossip协议管理成员关系、广播消息到整个集群


![GOSSIP](gossip.gif "gossip")

### 10-14

1. UiPath (Stadio, Fabric(process), Doc U)
2. 百度EasyDL (只提供访问接口, 没有模型设计)
3. 阿里PAI Stadio (数据预处理, 特征工程, ML)
4. DeepCognitonAI (无探测)
5. microsoft azure (ML有拖拽, DL以代码形式)
6. machinelearningforkids (没找到模型如何建立的, 但是训练后的模型可以在Scratch应用)
   - 从另一种方式教孩子学习AI
7. Machine Learning Playground (传统ML, 有意思, 自己创数据, 选择模型, 及时看到结果)

https://www.uipath.com/product/platform/rpa-ai-integration-with-ai-fabric

https://www.uipath.com/blog/easier-artificial-intelligence-starter-machine-learning-models

> While AI has become a shorthand for a range of big data, statistical, and predictive techniques, UiPath focuses on “practical AI,” applying AI in tangible scenarios that improve automation outcomes. We’ve built AI into every part of the UiPath Platform, from detecting objects using computer vision, to discovering automation opportunities, and managing and scaling AI. In turn, businesses are moving from simpler automation scenarios to more complex ones, which involve unstructured data, lots of variables, and uncertain outcomes.

> You don’t need to be a data scientist to use ML models
UiPath makes it easy for RPA developers and business users to benefit from AI and ML models without deep data science expertise through products such as UiPath AI Fabric and UiPath Document Understanding.

> With AI Fabric and Document Understanding, ML model retraining has been democratized for non-data scientists, enabling RPA developers and business users with domain-specific knowledge to easily mark up samples. Then they can use AI Fabric to retrain the model without needing to understand or think about feature engineering, hyperparameterization, or any other data scientist-specific knowledge.

Blue Prism
这是一款RPA软件，可为其提供灵活的数字化劳动力组织.它可以从 https://www.blueprism.com/下载.

Automation Anywhere
RPA的另一个重要工具是Automation Anywhere，它可以从 https://www.automationanywhere.com/

UiPath
这是一个RPA软件.它可以帮助组织有效地自动化流程.它可以从 https://www.uipath.com/

1. [machinelearningforkids](https://machinelearningforkids.co.uk/)

  数据标注一体话
  
2. [Machine Learning Playground](https://ml-playground.com)

  所见即所得

https://www.stevemurch.com/machine-learning-ai-for-kids-resources/2018/12

### 10-15 至 10-18

1. 成都出差

    - 参观活动会场
    - '爱阅读'书城查看儿童书籍, 优学派儿童产品了解
    - 签署战略协议

---------------------

## 19-23

### 10-19

1. consul集群设计思路

> GPU机器按区域划分, 每个区域下的所有GPU节点中选一个节点作为sever代理, 该区域其他节点作为client代理, 每个代理节点(GPU机器)都会启动若干个框架相关的服务, client节点将服务状态汇总所在区域的server代理, 所有区域server代理节点在选举出一个leader, 其他作为follower(保证不会应该单一server节点挂了,监控功能被废), 可能的问题: server节点理论上不能太多, 如果区域多了, 这个方案可能需要调整

![](consul-client-server.png)

https://github.com/sunweisheng/spring-cloud-example

```

                                          (wlan/lan)                    (wlan/lan)
                         +--------------+             +--------------+              +--------------+
                         |              | ----------> |              | -----------> |              |
                         |  Server:chi  |             |  Server:eta  |              | Server:sigma |
                         |              | <---------- |              | <----------- |              |
                         +--------------+             +--------------+              +--------------+
                           (follower)                    (leader)                      (follower)
                               ^
                               |
                               | (lan: gossip)
                               |
                               |
    +--------------------------|------------------------+  选择3个机器作为consul server模式
    |                                                   |
    |  +-------------------+     +-------------------+  |  按区域分配机器, 每个gpu机器节点启动consul client模式
    |  |  Client:001-001   |     |  Client:001-002   |  |
    |  |        |          |     |        |          |  |  每个gpu节点启动多个微服务(cv/nlp等)向client agent注册
    |  |  ai/cv/ml/rl/nlp  |     |  ai/cv/ml/rl/nlp  |  |
    |  |    (services)     |     |    (services)     |  |  client agent把消息同步到consul server端
    |  +-------------------+     +-------------------+  |
    |                                                   |  服务监控就可以统一做到server端
    |                +-------------------+              |
    |                |  Client:001-003   |              |
    |                |        |          |              |
    |                |  ai/cv/ml/rl/nlp  |              |
    |                |    (services)     |              |
    |                +-------------------+              |
    |                                                   |
    +---------------------------------------------------+
```

### 10-20

1. 模拟测试consul集群, 基本测试通, 但在测试选举leader时, server节点多会出现混乱

2. 真实环境部署consul server (http://116.85.5.40:8500/)

3. consul client真实环境尚没有测试(明天需要滴滴云开些账号)

4. consul monitor check 方案可行性研究(通过)

https://www.techdrabble.com/hashicorp/37-use-a-consul-watch-to-monitor-vault-seal-status

### 10-21

1. 手机短信发送测试 (筑望平台消息内容必须含有"【】验证码 "字符, 否则发不出去)
  - python脚本测试ok
  - (失败)尝试使用shell实现短息发送(docker consul没有python运行环境)
  - (方案1) 不使用docker中consul, 直接在使用原程序, 重新修改原逻辑. 缺点: 没有always restart能力
  - (方案2:采用) 做一个新consul docker含有python环境
  
2. client代理环境配置及问题:
  - (已解决)快照方式分配的机器, 解决nvidia驱动内置不配问题(WARNING! Diff between built and installed module!)
  - 框架服务crash状态触发的脚本执行存在问题


### 10-22

1. (已解决)failed inserting node: Error while renaming Node ID (node-id暂时使用固定TODO)

2. (已完成)构建带python环境的consul, 短信发送采用python3自带的request, 减少安装依赖

3. (已解决)ai服务注册方式由代码硬编码改为配置方式, 避免consul挂掉无法唤起ai服务注册

4. (未解决)检测异常服务, 监控脚本会多次被执行问题

    - 方案: 在监控脚本中增加判断是否为leader(1/3)的节点, 由leader发送消息避免重复 (未完成)

5. (未解决)所有节点运行一段时间, 概率出现所有节点都退出的问题

### 10-23

1. (已解决)检测异常服务, 监控脚本会多次被执行问题

2. (未解决)服务reload或者异常重启时, 所有watch监控的服务脚本会被触发(BUG)

    - 临时特殊处理一下, 待官网更新

    - issue [#7616](https://github.com/hashicorp/consul/pull/7616)
    
3. (已解决)所有节点启动过程会报服务check状态, 虽然服务是启动的, 但仍处在不可达状态, 导致误报

    - TODO: 触发异常处理脚本, 脚本中二次check服务状态, 防止上报错误状态信息.
    
4. (已完成)短信上报功能

    - TODO: 短信内容大小有限制,多个节点同时上报服务异常, 短信发不出去

-----------------

## 26-30

### 10-26

- (已解决)consul定时退出重启问题 (功能由定时check服务冲突导致)

### 10-27

- 增加自定义网络为空的错误提示 (推荐直接由前端直接判断)

- 增加consul记录用户训练错误信息 (暂没做清除机制, 会产生数据积累, 产品稳定后删除此功能)

- 在服务启动时, 同时启动调试环境下的www服务, 开发测试需要

- 增加用户训练异常信息发送邮件通知功能

- (未完成) ssd代码分析 (yatengLG/SSD-Pytorch 和 lufficc/SSD)

- https://towardsdatascience.com/review-ssd-single-shot-detector-object-detection-851a94607d11

- https://github.com/hashicorp/consul/issues/8687 (解决consul data modify trigger Consul 1.9)

### 10-28

- yatengLG/SSD-Pytorch (代码耦合严重,不适合模块化)

- 增加获取用户训练任务实际占用的GPU, 如果大于设置的阀值页面提示

- Redis消息添加过期时间(5天)

- 联调多输入层生成proto层顺序乱问题

### 10-29

- 分析ssds.pytorch 和 a-PyTorch-Tutorial-to-Object-Detection(代码单一,容易拆分)

- 定位自定义模型输入的层顺序问题(尚未解决,前端逻辑判断复杂)

- 分析生成自定义网络部分代码, 看是否能够节省内存

  - 在forward时是每个层对应一个独立变量, 成倍增大了内存使用(目前还没有解决方案)

- 任务2 连线不完整的情况下依旧可以进行训练
   - (新需求) tabpage跳转模型检查保存问题   

- 任务2 线性层/全连接层不能根据输出层自动调节，需要手动修改参数
   - (非问题)

- 任务2 在训练进行中选择暂停训练 再选择开始训练 系统显示bug
   - (前端) 暂时训练后"开始训练"按钮置灰

- 连续点击继续训练/暂停训练按钮会报bug, 提示框队列展示问题
   - (体验) 优先级低, 暂不解决

- 任务3 模型搭建完成之后训练进程不动了
   - (BUG) 已解决

- 删线逻辑有问题(多输入层, 如果删除连线再次连接, 生成proto层序列问题)
   - (前端) 优先级高, 难度有点大 **尚未解决**

### 10-30

SSD网络三种方案:

- (改动大)细分模块可配置SSD, 该方案实施难度大, 涉及到前端拖拽重新设计以及框架解析部分也重新构造

- (已实现)黑箱SSD模块,即SSD作为完整模块, 无配置不灵活, 且基础分类网络要符合一定条件(1024/512) 

- (竞赛)直接提供源码接口, 用户也不需要拖拽

<img src=ssd_drop.png width=400/>

-----------------

## 02-06

### 11-02

- 挑选下载国内垃圾数据集

- 垃圾分类可参考系统

  [智能语音垃圾分类系统](https://www.bilibili.com/video/BV12E411E7r3/?spm_id_from=trigger_reload)
    
  [智能快速分类垃圾箱](https://www.bilibili.com/video/BV1V7411c7Mr/?spm_id_from=333.788.videocard.2)
    
  [呆萌教授带学霸姐示范制作可分类垃圾箱](https://www.bilibili.com/video/BV1mx411T71s?from=search&seid=13070104863205387901) (推荐)
  
- 讨论教学与竞赛

### 11-03

- 垃圾数据集处理,训练

- 讨论需求


### 11-04

竞赛 (GPU/人):

- GPU机器申请(一人一机)

  目前只能采用人工手动方式, 运维人根据申请人的配置去滴滴云手动配置相应的GPU机器


- 产品设计

  - 低 1周 (垃圾分类)
  
  - 中 1周 (⻔禁系统)
  
  - 高


- AI软件功能控件 (低)

  - 数据标注 (前端)
  
     + 页面设计 (10分类 + 100张)
     
     + 数据存储

  - 分类模块 4周
     
     + 垃圾分类器
     
     + 通用分类器 (用户标记数据集 + 选择预制模型 + 训练 => 用户分类器)
  
  - 探测模块 4周
  
     + 人脸识别器 + 人脸库
     
     + 车牌识别器 + 字符提取
          
  - 自然语言 8周
  
     + QA (聊天问答)
  
  - 语音识别 10周

     + 语音转文字模块
      
     + 智能语音控制 AIOT
     
  - 机器学习 ?
     

- 模拟器 (中) ?

  - 模块关联
  
  - 模块参数配置

- 硬件集成 (高) ?

  - 配置系统

  - 嵌入式模型移植
  
  

垃圾分类评估预测模块并部署完成, notebook调试验证完成

## 11-05

### 调研开发板

#### 树莓派

   - https://item.jd.com/70026273258.html  (无摄像头)
   - https://item.jd.com/10020414798098.html (摄像头)
   - https://item.jd.com/69950397183.html#crumb-wrap


#### 安装

   - http://mathinf.com/pytorch/arm64/
   - https://discuss.pytorch.org/t/pytorch-1-3-wheels-for-raspberry-pi-python-3-7/58580/12
   - https://downloads.raspberrypi.org/raspios_lite_arm64/images/raspios_lite_arm64-2020-08-24
   
   
结论: 采用树莓派Pi4版本(高配1300$), 根据客服的意思, 可以自己安装需要的系统, 那么就可以交叉编译出pytorch移植到板子上(或者采用已经交叉编译后的程序移植).
   
   
### 垃圾分类时间计划

    
开发负责人| 开发用时 | 备注
:--: | :--: | :--
<img width=150/> | <img width=150/> | <img width=400/>
UI | 8天 | 2天(产品主题验证体验) + 6张 * 1天
前端 | 12天 | 6张 * 2天 | 依赖UI设计完
<br>框架 | <br>12天 | 1. 竞赛独立框架初级设计(3天) <br> 2. 微调预置三个模型以上的垃圾分类器, 尽可能区分出**好**,**中**,**差** (7天) <br> 3. 调试(2天)

## 11-06

### RACE(谐音锐思)竞赛平台框架

- 假设前提: 参加竞赛的用户所分配到的机器, 足以满足该用户所有操作要求.

- 开源框架集成
    
   + 通用分类: [pytorch lightning](https://github.com/PyTorchLightning/pytorch-lightning)
      - 代码模板设计(TODO)
        > 基于模板抽象出用户参数
      
   + 对象探测: [detectron2](https://github.com/facebookresearch/detectron2)
      - 代码模板设计(TODO)
        > 基于模板抽象出用户参数
      
   + 自然语言: [allennlp](https://github.com/allenai/allennlp)
      - 代码模板设计(TODO)
        > 基于模板抽象出用户参数
      
   + 语音识别: [speechbrain](https://speechbrain.github.io) 暂未开放
      - 代码模板设计(TODO)
        > 基于模板抽象出用户参数
        
   **考虑到健壮行, 还是将几个框架独立运行, 相互不干扰, 减少冲突**
        
- 框架数据存储

   + 根目录: `/raceai` 平台所有数据根位置ROOT
   
   + 框架代码: `{ROOT}/codes` 框架所有服务代码, 不同功能代码放置各自的子目录, 如:`/raceai/codes/hzcsrace`
   
   + 预置权重: `{ROOT}/ckpts` 预置模型权重, 不同类型的框架, 放置各自的子目录, 如:`/raceai/ckpts/cv`
   
   + 预置数据集: `{ROOT}/datasets` 竞赛项目暂时不需要预置数据集
   
   + 用户数据集: `{ROOT}/users/{userid}/datasets/{dbid}` TODO: 如果用户GPU使用时长结束, 数据会消失
   
   + 用户权重: `{ROOT}/users/{userid}/ckpts/{modelid}` TODO: 如果用户GPU时长结束, 数据会消失
   
   + 中间数据: `{ROOT}/users/{userid}/{projectid}`
   
   
- 框架基本功能

   + 上传下载: 适用于传递大文件, 比如数据集zip包, 模型权重等, 不考虑并发能力(基于假设前提)
   
   + 交互接口:
      - 训练: `/raceai/framework/fit`
        > 参数形式(TODO, 待基于模板抽象出参数之后再定义)
      
      - 测试: `/raceai/framework/test`
        > 参数形式(TODO, 待基于模板抽象出参数之后再定义)
      
   **异步通讯采用简单版的TCP端口号长连接方式(申请机器给的地址和端口)**
   
- Talenai教育平台打通

   + 自定义模型: TODO

   + 复用训练权重: 两个平台的使用框架不统一, 权重加载需额外处理 (TODO, 短期暂不考虑)

-------------

# 09-13

## 11-09

- 解决上周遗留的垃圾分类对实际数据正确率低的问题

- H5的演示demo,解决GPU内存不释放问题(临时采用启动子进程去执行, 执行后子进程销毁回收资源) 

- 框架部署
   只部署分类和探测, 其他框架有需求时在加入.
   
   **问题**
     - 对预测类任务测试性能分析, 采用docker方式启动太慢(等待时间长), 针对预测任务采用不启动docker方式
     - 不采用docker方式, 将会增加资源释放难度, 尤其是cuda内存资源(实验验证, 即使调用了相关接口, 系统不会自动回收cuda)
   
- 分类模型参数定义(采用类名字符串+类参数组织)

```json
{
    "userid": "1",
    "projectid": "123456",
    "task": "cls",
    "cfg": {
        ...
        "optimizer": {
            "class_name": "torch.optim.Adam",
            "params": {
                "lr": "0.001",
                "weight_decay": 0.001
            }
        },
        ...
    }
}
```

"/raceai/framework/test"

```json
{
    "userid": "1",
    "projectid": "123456",
    "task": "cls",
    "cfg": {
        "data": {
            "path": "oss://xxx"   // data:image/jpeg;base64,
        }
        "model": {
            "backbone": {
                "class_name": "raceai.models.vgg16",
                "params": {
                    "num_classes": 10,
                    "pretrained": true
                }
            },
            "ckpt_path": "path/to/ckpt"
        }
    }
}
```

"/raceai/framework/fit"

```json
{
    "userid": "1",
    "projectid": "123456",
    "task": "cls",
    "cfg": {
        "data": {
            "path": "",
            "name": "test"
        },
        "trainer": {
            "class_name": "a.b.c",
            "params": {
                
            }
            "max_epochs": 100,
        },
        "model": {
            "backbone": {
                "class_name": "raceai.models.classifier",
                "params": {
                    "code": "",
                }
            },
            "ckpt_path": "path/to/ckpt"
        },
        "optimizer": {
            "class_name": "torch.optim.Adam",
            "params": {
                "lr": "0.001",
                "weight_decay": 0.001
            }
        },
        "scheduler": {
            "class_name": "torch.optim.lr_scheduler.ReduceLROnPlateau",
            "params": {
                "mode": "min",
                "factor": 0.1,
                "patience": 5
            }
        },
        "criterion": {
            "class_name": "torch.nn.CrossEntropyLoss",
            "params": {
                "reduction": "mean"
            }
        }
    }
}
```

Intersection over Union (IOU)              
非极大值抑制 NMS
ROI Align
Ground Truth边框 Region proposal边框
尺度因子
R-CNN利用候选区域的方法（Region Proposal) 选出2000个独立的候选区域  svm分类器 回归器

1. Region of Interest – ROI

hard negative mining

> 在目标检测中我们会事先标记好ground_truth，接下来在图片中随机提取一系列sample，与ground_truth
重叠率IoU超过一定阈值的(比如0.5)，则认为它是positive sample，否则为negative sample，
考虑到实际负样本数>>正样本数，我们为了避免network的预测值少数服从多数而向负样本靠拢，取正样本数：负样本数大约为1:3，
显而易见，用来训练网络的负样本为提取的负样本的子集，那么，我们当然选择负样本中容易被分错类的困难负样本来进行网络训练啰。

SIFT:
> 在一定的范围内，无论物体是大还是小，人眼都可以分辨出来。然而计算机要有相同的能力却不是那么的容易，在未知的场景中，计算机视觉并不能提供物体的尺度大小，其中的一种方法是把物体不同尺度下的图像都提供给机器，让机器能够对物体在不同的尺度下有一个统一的认知。在建立统一认知的过程中，要考虑的就是在图像在不同的尺度下都存在的特征点。

暴力穷举法 --> selective search(颜色, 纹理, 边缘 启发式搜索算法)

## 11-10

- 微服务(微信H5)应用, 修改所有任务启动方式(采用独立程序启动,方便释放cuda内存), 并添加接口时间消耗装饰

## 11-11

- 竞赛分类预测框架代码基本coding完成, 调试OK

- UI垃圾分类产品任务细节沟通及UI开发任务启动

## 11-12

- 图形形状相关

   - (分类)图形数据集 https://www.kaggle.com/smeschke/four-shapes

   - (探测)图形 https://github.com/AbdelrahmanRadwan/object-detection

   - (探测)图形数据集生成 https://github.com/matterport/Mask_RCNN/blob/master/samples/shapes/shapes.py
   
   **都不太符合我们的需求**
   

    

https://www.kaggle.com/artgor/object-detection-with-pytorch-lightning

## 11-13

- 竞赛基础场景
  
  - AI找茬(找出同类, 对常见的小的icon进行混合分两组, 两组必有一个icon是相同的)
    
    属于实例探测范畴, 具体场景设计: 
      
      1. 分两组, 一组8个元素(如 飞机, 蜘蛛, 帽子等任何icon)
      2. 这两个组只有一个元素是相同的(比如,都含有蜘蛛)
      3. 组内所有元素都经过随机旋转, 随机缩放(制造分辨的难度, 查找更困难)
      4. 学生和AI同时**找茬(同类)**, 谁先找到谁胜. (前端设计难度可能会很大, 尤其是学生点击元素)
      5. 这个场景学到什么: (1) 目标检测 (2) 识别准确度和识别速度的平衡
      
    合适的数据集没找到, 但是可以从手机或电脑的一些图标(icon)获取, 自己设计数据集.
      
      
  - 口罩探测(其实是在人脸探测基础上, 再识别人脸是否有口罩)
    
     如何设计暂时没想好, 数据已经找到(https://github.com/AIZOOTech/FaceMaskDetection)
     
     相同的场景如: 车牌探测(在车牌识别基础上, 再将车牌号摘出来, 进而匹配数据库,判断是否为外来车)
     
     学到了什么: 一个复杂的系统, 可以由多个简单功能模块进行组合 (可以设计一些if/then/else的简单逻辑模块)
     
     
- 竞赛探测预测框架coding (还需要完善)

In [ ]:
https://github.com/aakarsh7599/Text-Detection-using-Detectron2
https://github.com/AIZOOTech/FaceMaskDetection
https://towardsdatascience.com/how-i-learned-my-computer-to-play-spot-it-using-opencv-and-deep-learning-ad1f017a3ec3
https://github.com/henniedeharder/spotit/tree/master/DeepLearningSpotIt/computervision
https://dinone.kf5.com/hc/kb/article/1366537/

-----------------------

## 11-16 - 11-20

### 11-16

- 熟悉树莓派开发板

- 竞赛框架参数配置修改部分


### 11-17

- [PyTorch packages for ARM64](http://mathinf.com/pytorch/arm64/)

### 11-18

- 板上探测程序能够执行, 速度很慢10s以上 (可能OpenBLAS Warning : Detect OpenMP Loop...警告有关)

  - OpenBLAS编译问题仍在定位
  
- 研究yolov5版本, 并移植到开发版 (未完成)


https://blog.csdn.net/leviopku/article/details/82660381

[yolov3](https://www.yinxiang.com/everhub/note/1c8b7975-72fd-4bbf-abd2-37a9000e0d0a)

[yolov5](https://chtseng.wordpress.com/2020/07/03/yolov5%E5%9C%A8%E6%A8%B9%E8%8E%93%E6%B4%BE%E4%B8%8A%E7%9A%84%E6%B8%AC%E8%A9%A6/)

### 11-19

- 定位OpenBLAS Warning问题, 属于代码工程编译问题, github已有人提出bug

  - [Report error when ATEN_THEADING is OMP and USE_OPENMP is turned off.](https://github.com/pytorch/pytorch/pull/40146)
  
  - 由于bug提出不久, 对新版本的torch的armv8版本还没有修复
  
  - 尝试使用低版本的torch(1.6,17两个版本), 不再出现警告问题, 但是探测速度巨慢(OpenMP这个并行库还是有必要)
  
  - 由源码重新交叉编译torch, 尚没有结果(大概需要5小时以上的编译过程)
  
- 最新yolov5的代码研究及应用, 需要对数据集重新处理(未完成)


--------

## 11-23 - 11-27

### 11-23

- 调试3个gpio程序(led灯, lcd显示屏, 蜂鸣器)

- 调试摄像头(未完成)

### 11-24

- aarch64: "ERROR: mmal not found" (64系统下mmal视频编解码库无法编译使用)
    - kernel中就没加相关模块
    - 部分视频帧相关接口不能使用，如opencv2

- 寻找使用matplotlib+cv2解决camera数据帧(效率一般)

### 11-25

In [ ]:
- yolov5抓取camera数据探测

    - 调试yolov5 + camera + 车牌 (效果不好, 速度2.5s左右)

    - 调试yolov5 + camera + 人脸 (效果一般, 速度2.5s左右)

### 11-26

In [ ]:
- 测试方案: 将探测模型转为onnx放入开发板上直接执行
    
    - 快几十毫秒, 几乎没有性能提升
    
- 整理一下gpio的样例代码并提交

- 优化raceai(锐思)平台

    - 添加函数/类注册装饰接口, 大大代码简化
    - 配置微调, 尽兼容RestAPI及命令行两种方式